In [1]:
# jack tarricone
# february 18th, 2021 

# amended dnppy.radar.create_header script to work for current UAVSAR annotation file format
# it parses the annoation file by finding the line of your selected search phrase is in
# it then searches that line by number of characters, and creates an object out of that character selection
# does this for data type, lat, lon, lines, samples (add pixel size)
# it then inserts these information into the .hdr file


# feb 26th, 2021
# add in the loop at the end to load in the .grd files (bc of thier newly created headers) to python
# use gdal translate to convert them to .tiff
# save them in the same place as you input pathway
# right now .int data type has to be manually changed to 6

# march 3rd
# .int loop added, still only works in ENVI
# two different raster writes with correct data type added (4 float32, 6 Cfloat32)

# march 10th
# things to add to make this robust/complete: 

    # 1. add pixel size searching variable in case that changes (is this needed HP?)
    
    # 2. add functionality to break the .int.grd into a real and imaginary seperate rasters
        # right now it's in the correct form, but can only be read in by ENVI and not R, Python Qgis, arcgis
        
    # 3. add a step after doing the gdal transform to do a quick QA/QC crop on the rasters
        # I was able to do this in R, where you take the .cor raster and create an extent mask with it
        # i'm very sure it's possible in python, just and i envision the steps going like this :
            # once .tiffs created, pull the .cor raster back into python using rasterio
            # convert data outside of study area to NA (in R the value was -10000)
            # create mask layer from the update .cor raster
            # loop back through the other rasters we just created, crop them by extent, and resave them
        
# write pixel shift functionality

In [2]:
# beforing importing, you'll need to install these packages to the conda environment the notebook is running on

import os
import re
import codecs
import rasterio
import numpy
import matplotlib
from matplotlib import pyplot
import glob
import subprocess
from osgeo import gdal

In [3]:
# folder is path to a folder with an .ann (or .txt) and .grd files (.amp1, .amp2, .cor, .unw, .int)

def uavsar_tiff_convert(folder):
    """
    Builds a header file for the input UAVSAR .grd file,
    allowing the data to be read as a raster dataset.
    :param folder:   the folder containing the UAVSAR .grd and .ann files
    """


    os.chdir(folder)
    int_file = glob.glob(os.path.join(folder, 'int.grd'))

    # Empty lists to put information that will be recalled later.
    Lines_list = []
    Samples_list = []
    Latitude_list = []
    Longitude_list = []
    Files_list = []

    # Step 1: Look through folder and determine how many different flights there are
    # by looking at the HDR files.
    for files in os.listdir(folder):
        if files [-4:] == ".grd":
            newfile = open(files[0:-4] + ".hdr", 'w')
            newfile.write("""ENVI
description = {DESCFIELD}
samples = NSAMP
lines = NLINE
bands = 1
header offset = 0
data type = DATTYPE
interleave = bsq
sensor type = UAVSAR L-Band
byte order = 0
map info = {Geographic Lat/Lon, 
            1.000, 
            1.000, 
            LON, 
            LAT,  
            0.0000555600000000, 
            0.0000555600000000, 
            WGS-84, units=Degrees}
wavelength units = Unknown
                """
                          )
            newfile.close()
            if files[0:18] not in Files_list:
                Files_list.append(files[0:18])

    #Variables used to recall indexed values.
    var1 = 0

    #Step 2: Look through the folder and locate the annotation file(s).
    # These can be in either .txt or .ann file types.
    for files in os.listdir(folder):
        if Files_list[var1] and files[-4:] == ".txt" or files[-4:] == ".ann":
            #Step 3: Once located, find the info we are interested in and append it to
            # the appropriate list. We limit the variables to <=1 so that they only
            # return two values (one for each polarization of
            searchfile = codecs.open(files, encoding = 'windows-1252', errors='ignore')
            for line in searchfile:
                if "Ground Range Data Latitude Lines" in line:
                    Lines = line[65:70]
                    print(Lines)
                    if Lines not in Lines_list:
                        Lines_list.append(Lines)

                elif "Ground Range Data Longitude Samples" in line:
                    Samples = line[65:70]
                    print(Samples)
                    if Samples not in Samples_list:
                        Samples_list.append(Samples)

                elif "Ground Range Data Starting Latitude" in line:
                    Latitude = line[65:85]
                    print(Latitude)
                    if Latitude not in Latitude_list:
                        Latitude_list.append(Latitude)

                elif "Ground Range Data Starting Longitude" in line:
                    Longitude = line[65:85]
                    print(Longitude)
                    if Longitude not in Longitude_list:
                        Longitude_list.append(Longitude)
    
                        
                 
            #Reset the variables to zero for each different flight date.
            var1 = 0
            searchfile.close()


    # Step 3: Open .hdr file and replace data for all type 4 (real numbers) data
    # this all the .grd files expect for .int
    for files in os.listdir(folder):
        if files[-4:] == ".hdr":
            with open(files, "r") as sources:
                lines = sources.readlines()
            with open(files, "w") as sources:
                for line in lines:
                    if "data type = DATTYPE" in line:
                        sources.write(re.sub(line[12:19], "4", line))
                    elif "DESCFIELD" in line:
                        sources.write(re.sub(line[15:24], folder, line))
                    elif "lines" in line:
                        sources.write(re.sub(line[8:13], Lines, line))
                    elif "samples" in line:
                        sources.write(re.sub(line[10:15], Samples, line))
                    elif "LAT" in line:
                        sources.write(re.sub(line[12:15], Latitude, line))
                    elif "LON" in line:
                        sources.write(re.sub(line[12:15], Longitude, line))
                    else:
                        sources.write(re.sub(line, line, line))
    
    # Step 3: Open .hdr file and replace data for .int file date type 6 (complex)                 
    for files in os.listdir(folder):
        if files[-8:] == ".int.hdr":
            with open(files, "r") as sources:
                lines = sources.readlines()
            with open(files, "w") as sources:
                for line in lines:
                    if "data type = 4" in line:
                        sources.write(re.sub(line[12:13], "6", line))
                    elif "DESCFIELD" in line:
                        sources.write(re.sub(line[15:24], folder, line))
                    elif "lines" in line:
                        sources.write(re.sub(line[8:13], Lines, line))
                    elif "samples" in line:
                        sources.write(re.sub(line[10:15], Samples, line))
                    elif "LAT" in line:
                        sources.write(re.sub(line[12:15], Latitude, line))
                    elif "LON" in line:
                        sources.write(re.sub(line[12:15], Longitude, line))
                    else:
                        sources.write(re.sub(line, line, line))
                        
    
    # Step 4: Now we have an .hdr file, the data is geocoded and can be loaded into python with rasterio
    # once loaded in we use gdal.Translate to convert and save as a .tiff
    
    # Dan, if you wanted to convert to UTM, it would probably be best in this step
    # but also possibly when making the header, where lat/lon is specified 
    
    data_to_process = glob.glob(os.path.join(folder, '*.grd')) # list all .grd files
    for data_path in data_to_process: # loop to open and translate .grd to .tiff, and save .tiffs using gdal
        raster_dataset = gdal.Open(data_path, gdal.GA_ReadOnly)
        raster = gdal.Translate(os.path.join(folder, os.path.basename(data_path) + '.tiff'), raster_dataset, format = 'Gtiff', outputType = gdal.GDT_Float32)
    
    # Step 5: Save the .int raster, needs seperate save because of the complex format
    data_to_process = glob.glob(os.path.join(folder, '*.int.grd')) # list all .int.grd files (only 1)
    for data_path in data_to_process:
        raster_dataset = gdal.Open(data_path, gdal.GA_ReadOnly)
        raster = gdal.Translate(os.path.join(folder, os.path.basename(data_path) + '.tiff'), raster_dataset, format = 'Gtiff', outputType = gdal.GDT_CFloat32)

    print(".tiffs have been created")
    return



In [5]:
folder = "/Volumes/JT/projects/uavsar/sierra" 
uavsar_tiff_convert(folder)

15420
6485

37.838637960000000  
-119.167643760000004
.tiffs have been created


In [12]:
# list all .tiff raster files we just created
raster_list = glob.glob(os.path.join(folder, '*.tiff'))
print(raster_list)

['/Volumes/JT/projects/uavsar/rockmt_2021/rockmt_32109_21017-013_21021-001_0012d_s01_L090HH_01.hgt.grd.tiff', '/Volumes/JT/projects/uavsar/rockmt_2021/rockmt_32109_21017-013_21021-001_0012d_s01_L090HH_01.unw.grd.tiff', '/Volumes/JT/projects/uavsar/rockmt_2021/rockmt_32109_21017-013_21021-001_0012d_s01_L090HH_01.amp2.grd.tiff', '/Volumes/JT/projects/uavsar/rockmt_2021/rockmt_32109_21017-013_21021-001_0012d_s01_L090HH_01.amp1.grd.tiff', '/Volumes/JT/projects/uavsar/rockmt_2021/rockmt_32109_21017-013_21021-001_0012d_s01_L090HH_01.cor.grd.tiff', '/Volumes/JT/projects/uavsar/rockmt_2021/rockmt_32109_21017-013_21021-001_0012d_s01_L090HH_01.int.grd.tiff']


RuntimeError: not a string